# Introducción

Este cuaderno documenta la recopilación automatizada de datos sobre apartamentos en arriendo de Ibagué, utilizando técnicas de web scraping. La información obtenida servirá como base para el desarrollo de modelos de machine learning enfocados en la predicción de precios de arriendo, facilitando así el análisis y la toma de decisiones en el mercado inmobiliario.

# Preparación del entorno de trabajo

Se realiza la carga de las librerías necesarias:

In [124]:
# web scraping
import requests
import lxml.html
import json

# procesamiento
import pandas as pd
import math

# herramientas adicionales
import csv
import re
from datetime import datetime

Definición de fecha actual para la identificación de las descargas:

In [125]:
fecha_actual = datetime.now().strftime('%Y%m%d')

# Descarga de información

En esta sección se realiza la descarga de información mediante el procedimiento de web scraping. Posteriormente, se realiza la depuración de la información descargada.

Se inicia creando el objeto `enlace`, que contiene el enlace desde el cuál se realizará la predicción.

In [126]:
enlace='https://www.fincaraiz.com.co/arriendo/apartamentos/ibague/tolima'

Se inicia determinando el número de páginas disponibles, lo que sugiere un número aproximado del total de apartamentos en arriendo.

In [127]:
# indicación para guardar el HTML descargado en un archivo
guardar_debug = True

# definición de un agente de usuario que simula a un navegador real
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/126.0.0.0 Safari/537.36"
    )
}

# se realiza la solicitud de la página web 
# (tiempo máximo de 30 segundos)
# si la respuesta tiene un error HTTP (404, 500, etc.),
# raise_for_status() detiene la solicitud
resp = requests.get(enlace, headers=headers, timeout=30)
resp.raise_for_status()

# si guardar_debug es True, se almacena el HTML en un archivo local "debug.html"
if guardar_debug:
    with open("debug.html", "w", encoding="utf-8") as f:
        f.write(resp.text)

# conversión del contenido de la respuesta en un árbol DOM con lxml,
# lo que permite navegar el HTML con expresiones XPath
arbol = lxml.html.fromstring(resp.content)

# uso de regex sobre el HTML para localizar un texto del estilo: "Mostrando 1 - 21 de N".
m = re.search(r"Mostrando\s+\d+\s*-\s*\d+\s*de\s*([\d\.]+)", resp.text, re.IGNORECASE)
if m:
    # Se extrae el número (grupo 1), se eliminan los separadores de miles (puntos)
    # y se convierte a entero.
    numero_apartamentos = int(m.group(1).replace('.', ''))
    print(f"Total de apartamentos: {numero_apartamentos}")
else:
    # Segundo intento (fallback): buscar en el árbol DOM el primer <div>
    # que contenga la palabra "Mostrando". De ese texto se intenta extraer "de N".
    texto_mostrando = arbol.xpath('normalize-space(string(//div[contains(., "Mostrando")][1]))')
    m2 = re.search(r"de\s*([\d\.]+)", texto_mostrando)
    if m2:
        numero_apartamentos = int(m2.group(1).replace('.', ''))
        print(f"Total de apartamentos: {numero_apartamentos}")
    else:
        # Si no se pudo extraer el número con ninguno de los métodos, se lanza un error.
        raise ValueError("No fue posible extraer el total de apartamentos.")

# se cálcula el número de páginas a navegar
numero_paginas = math.ceil(numero_apartamentos / 21)
print(f"Total de páginas: {numero_paginas}")

Total de apartamentos: 169
Total de páginas: 9


A continuación, se realiza la recolección de los enlaces de cada uno de los apartamentos:

In [128]:
# creación de vector con los enlaces de los apartamentos
enlaces_apartamentos = []
# iteración sobre las páginas
for pagina in range(1, numero_paginas + 1):
    # configuración del seguimiento visual
    progreso = pagina / numero_paginas
    porcentaje = progreso * 100
    barra = '█' * int(porcentaje // 2) + '-' * (50 - int(porcentaje // 2))
    print(f'\rProcesando página {pagina}/{numero_paginas} |{barra}| {porcentaje:.2f}%', end='')
    
    # definición del enlace de la página
    enlace_pagina = 'https://www.fincaraiz.com.co/arriendo/apartamentos/ibague/tolima/pagina' + str(pagina)
    
    try:
        # solicitud a la página
        respuesta_pagina = requests.get(enlace_pagina, headers=headers, timeout=30)
        respuesta_pagina.raise_for_status()
        arbol_pagina = lxml.html.fromstring(respuesta_pagina.content)
        
        # XPath más robusto y flexible - buscar todos los enlaces de apartamentos
        xpath = '//a[contains(@href, "/apartamento")]'
        enlaces = arbol_pagina.xpath(xpath)
        
        # Si no encuentra nada con el XPath principal, intentar alternativas
        if not enlaces:
            # XPath alternativo más general
            xpath_alt = '//a[contains(@href, "apartamento")]'
            enlaces = arbol_pagina.xpath(xpath_alt)
        
        # Si aún no encuentra nada, intentar buscar por patrones en el texto
        if not enlaces:
            xpath_texto = '//a[contains(text(), "Apartamento") or contains(@href, "ibague")]'
            enlaces = arbol_pagina.xpath(xpath_texto)
        
        # Debugging para ver qué está pasando
        print(f"\nPágina {pagina}: {len(enlaces)} enlaces encontrados")
        
        # procesar los enlaces encontrados
        for enlace in enlaces:
            href = enlace.get('href')
            if href and '/apartamento' in href:
                # asegurar que el enlace sea relativo (sin dominio)
                if href.startswith('/'):
                    enlaces_apartamentos.append(href)
                elif 'fincaraiz.com.co' in href:
                    # extraer solo la parte relativa del enlace
                    href_relativo = href.split('fincaraiz.com.co')[-1]
                    enlaces_apartamentos.append(href_relativo)
                
                # exportación incremental
                with open(f'{fecha_actual}_enlaces_apartamentos.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
                    escritor = csv.writer(archivo_csv)
                    for enlace_guardar in enlaces_apartamentos:
                        escritor.writerow([enlace_guardar])
    
    except requests.exceptions.RequestException as e:
        print(f"\nError al acceder a la página {pagina}: {e}")
        continue
    except Exception as e:
        print(f"\nError inesperado en página {pagina}: {e}")
        continue

print()  # para salto de línea después de la barra de progreso

# filtra los enlaces que no cumplen con el patrón establecido
pat = re.compile(r"^/apartamento-en-arriendo-[a-z0-9-]*ibague/\d+$")
enlaces_apartamentos = [u for u in enlaces_apartamentos if pat.match(u)]

print(f'Se descargaron {len(enlaces_apartamentos)} enlaces de apartamentos en arriendo.')

Procesando página 1/9 |█████---------------------------------------------| 11.11%
Página 1: 63 enlaces encontrados
Procesando página 2/9 |███████████---------------------------------------| 22.22%
Página 2: 64 enlaces encontrados
Procesando página 3/9 |████████████████----------------------------------| 33.33%
Página 3: 64 enlaces encontrados
Procesando página 4/9 |██████████████████████----------------------------| 44.44%
Página 4: 64 enlaces encontrados
Procesando página 5/9 |███████████████████████████-----------------------| 55.56%
Página 5: 63 enlaces encontrados
Procesando página 6/9 |█████████████████████████████████-----------------| 66.67%
Página 6: 64 enlaces encontrados
Procesando página 7/9 |██████████████████████████████████████------------| 77.78%
Página 7: 64 enlaces encontrados
Procesando página 8/9 |████████████████████████████████████████████------| 88.89%
Página 8: 64 enlaces encontrados
Procesando página 9/9 |██████████████████████████████████████████████████| 100.0

Antes de continuar, se hace la revisión de los enlaces para evitar que se exporten duplicados. Una vez hecha, se hace la exportación:

In [129]:
# eliminación de duplicados en la lista de enlaces
enlaces_apartamentos = list(dict.fromkeys(enlaces_apartamentos))

# exportación de la lista sin duplicados
with open(f'{fecha_actual}_enlaces_apartamentos.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    escritor = csv.writer(archivo_csv)
    for enlace in enlaces_apartamentos:
        escritor.writerow([enlace])

# impresión del resultado para seguimiento
print(f'Se eliminaron duplicados. Ahora hay {len(enlaces_apartamentos)} enlaces únicos.')

Se eliminaron duplicados. Ahora hay 120 enlaces únicos.


De esta forma, se procede a construir la base de datos con la información encontrada en cada enlace:

In [130]:
datos_apartamentos = pd.DataFrame()
for enlace_apartamento in enlaces_apartamentos:
    # seguimiento visual del progreso en la iteración de los enlaces
    indice_actual = enlaces_apartamentos.index(enlace_apartamento) + 1
    total_enlaces = len(enlaces_apartamentos)
    progreso = indice_actual / total_enlaces
    porcentaje = progreso * 100
    barra = '█' * int(porcentaje // 2) + '-' * (50 - int(porcentaje // 2))
    print(f'\rProcesando enlace {indice_actual}/{total_enlaces} |{barra}| {porcentaje:.2f}%', end='')
    
    # construcción del enlace y solicitud a la página
    enlace_completo = 'https://www.fincaraiz.com.co' + enlace_apartamento
    
    try:
        respuesta_apartamento = requests.get(enlace_completo, headers=headers, timeout=30)
        respuesta_apartamento.raise_for_status()
        arbol_apartamento = lxml.html.fromstring(respuesta_apartamento.content)
    
        # definición de las columnas requeridas
        columnas = [
            'Enlace', 'Precio', 'Baños', 'Área Construida', 'Antigüedad', 'Habitaciones', 'Parqueaderos',
            'Área Privada', 'Estrato', 'Administración', 'Piso N°', 'Acepta mascotas',
            'Documentación requerida', 'Estado', 'Cantidad de pisos', 'Contrato Mínimo',
            'Latitud', 'Longitud'
            ]

        # inicialización de diccionario para almacenar los valores extraídos
        valores = {col: None for col in columnas}
        valores['Enlace'] = enlace_completo

        # extracción del precio (diferentes tipos de xpath)
        precio_apartamento = None
        xpaths_precio = [
            '//span[contains(@class, "price")]//text()',
            '//div[contains(@class, "price")]//text()',
            '//*[contains(text(), "$") and contains(text(), "000")]//text()',
            '//p[contains(text(), "$")]//text()',
            '//span[contains(text(), "$")]//text()'
            ]

        for xpath_precio in xpaths_precio:
            try:
                precio_elements = arbol_apartamento.xpath(xpath_precio)
                if precio_elements:
                    # búsqueda del elemento que contenga un precio válido
                    for precio_elem in precio_elements:
                        if isinstance(precio_elem, str) and '$' in precio_elem and any(char.isdigit() for char in precio_elem):
                            precio_apartamento = precio_elem.strip()
                            break
                    if precio_apartamento:
                        break
            except:
                continue
        
        valores['Precio'] = precio_apartamento

        # extracción de características 
        scripts = arbol_apartamento.xpath('//script[contains(text(), "technicalSheet")]')
        if scripts:
            script_text = scripts[0].text_content().strip()
    
            # Extraer el JSON del script
            json_start = script_text.find('{')
            json_end = script_text.rfind('}') + 1
            json_str = script_text[json_start:json_end]
    
            try:
                data = json.loads(json_str)
                technical_sheet = data['props']['pageProps']['data']['technicalSheet']
        
                for item in technical_sheet:
                    field = item['field']
                    value = item['value']
                    text = item['text']
            

                    # mapeo a nuestras columnas
                    if field == 'bathrooms':
                        valores['Baños'] = value
                    elif field == 'bedrooms':
                        valores['Habitaciones'] = value
                    elif field == 'm2Built':
                        valores['Área Construida'] = value
                    elif field == 'm2apto':
                        valores['Área Privada'] = value
                    elif field == 'stratum':
                        valores['Estrato'] = value
                    elif field == 'floor':
                        valores['Piso N°'] = value
                    elif field == 'constructionYear':
                        valores['Antigüedad'] = value
                    elif field == 'construction_state_name':
                        valores['Estado'] = value
                    elif field == 'garage':
                        valores['Parqueaderos'] = value
                    elif field == 'commonExpenses':
                        valores['Administración'] = value
                    elif field == 'floorsAmount':
                        valores['Cantidad de pisos'] = value
                    elif field == 'allowPets':
                        valores['Acepta mascotas'] = value
                    elif field == 'minimumContract':
                        valores['Contrato Mínimo'] = value
        
                # Extraer coordenadas
                lat = data['props']['pageProps']['data'].get('latitude')
                lon = data['props']['pageProps']['data'].get('longitude')
                if lat:
                    valores['Latitud'] = lat
                if lon:
                    valores['Longitud'] = lon
            
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error al procesar JSON: {e}")
        
        # adición de la fila al cuadro
        datos_apartamentos = pd.concat([datos_apartamentos, pd.DataFrame([valores])], ignore_index=True)
        
    except requests.exceptions.RequestException as e:
        print(f"\nError al acceder al enlace {enlace_completo}: {e}")
        continue

print()  # salto de línea después de la barra de progreso

# exportación de la base de datos
if not datos_apartamentos.empty:
    datos_apartamentos.to_csv(f'{fecha_actual}_apartamentos_ibague.csv', index=False, encoding='utf-8')
    print(f'Se exportaron {len(datos_apartamentos)} registros de apartamentos.')
    print(f'Archivo guardado como: {fecha_actual}_apartamentos_ibague.csv')
else:
    print('No se encontraron datos para exportar.')    

Procesando enlace 120/120 |██████████████████████████████████████████████████| 100.00%
Se exportaron 120 registros de apartamentos.
Archivo guardado como: 20250912_apartamentos_ibague.csv


Se hace una revisión rápida del resultado:

In [131]:
datos_apartamentos.sample(10, random_state=42)

,Enlace,Precio,Baños,Área Construida,Antigüedad,Habitaciones,Parqueaderos,Área Privada,Estrato,Administración,Piso N°,Acepta mascotas,Documentación requerida,Estado,Cantidad de pisos,Contrato Mínimo,Latitud,Longitud
44,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 529.000,2,49 m2,1 a 8 años,2,,,3,$ 171.000,,,None,Usado,,,4.449719,-75.143251
47,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 3.000.000,3,200.00 m2,16 a 30 años,4,,196.00 m2,3,,,,None,,,,4.440993,-75.226098
4,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 616.000,1,40 m2,1 a 8 años,2,,35 m2,3,$ 134.000,,,None,Usado,,,4.419924,-75.170624
55,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.500.000,3,120.00 m2,16 a 30 años,3,1,,3,,4,,None,,,,4.439575,-75.228605
26,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.400.000,2,75 m2,16 a 30 años,3,,75 m2,2,,4,,None,Usado,,,4.446831,-75.194156
64,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 800.000,1,90 m2,1 a 8 años,3,,,3,,1,,None,Usado,,,4.437587,-75.167863
73,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.950.000,2,106.00 m2,,3,1,106.00 m2,110,,,,None,,,,4.437001,-75.206974
10,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 615.000,2,56 m2,9 a 15 años,3,1,50 m2,3,$ 185.000,,,None,Usado,,,4.444200,-75.166085
40,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.100.000,2,57 m2,1 a 8 años,2,,,3,,5,,None,Usado,,,4.418958,-75.170131
107,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.250.000,1,38 m2,9 a 15 años,1,1,38 m2,4,,,,None,Usado,,,4.436552,-75.205586


Se exporta la base resultante:

In [132]:
datos_apartamentos.to_csv(f'{fecha_actual}_apartamentos_ibague.csv', index=False)

# Limpieza de datos

Se realiza la limpieza de datos para que correspondan al formato correcto. Inicialmente, se revisa la estructura de la base:

In [133]:
datos_apartamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Enlace                   120 non-null    object 
 1   Precio                   120 non-null    object 
 2   Baños                    120 non-null    object 
 3   Área Construida          120 non-null    object 
 4   Antigüedad               120 non-null    object 
 5   Habitaciones             120 non-null    object 
 6   Parqueaderos             120 non-null    object 
 7   Área Privada             120 non-null    object 
 8   Estrato                  120 non-null    object 
 9   Administración           120 non-null    object 
 10  Piso N°                  120 non-null    object 
 11  Acepta mascotas          120 non-null    object 
 12  Documentación requerida  0 non-null      object 
 13  Estado                   120 non-null    object 
 14  Cantidad de pisos        1

Todos las variables, con la excepción de la latitud y la longitud, aparecen como tipo texto. Ya que hay variables que son de tipo numéricas, se revisa cada una para ver cómo están. Antes de empezar, se eliminan las variables que no aportan información y se estandarizan los valores nulos.

## Eliminación de variables sin información

Se identifican las variables que solo tienen un datos, por lo que no aportan información:

In [134]:
columnas_unico_valor = [col for col in datos_apartamentos.columns if datos_apartamentos[col].nunique(dropna=False) == 1]
datos_apartamentos[columnas_unico_valor]

,Acepta mascotas,Documentación requerida,Contrato Mínimo
0,,None,
1,,None,
2,,None,
3,,None,
4,,None,
...,...,...,...
115,,None,
116,,None,
117,,None,
118,,None,


De esta forma, se eliminan:

In [135]:
datos_apartamentos = datos_apartamentos.drop(columns=columnas_unico_valor)

## Valores nulos

Aquellos valores con valor nulo se estandarizan para que tengan una misma forma.

In [136]:
datos_apartamentos = datos_apartamentos.replace(['¡Pregúntale!', '', 'None', '...', '<NA>'], pd.NA)

## Identificador

Se construye un identificador a partir del enlace del apartamento:

In [137]:
# Extraer el identificador del enlace después del último "/"
datos_apartamentos['Identificador'] = datos_apartamentos['Enlace'].str.split('/').str[-1]

# muestra de 10 filas con las columnas 'Enlace' e 'Identificador'
datos_apartamentos[['Enlace', 'Identificador']].sample(10, random_state=42)

,Enlace,Identificador
44,https://www.fincaraiz.com.co/apartamento-en-ar...,191707560
47,https://www.fincaraiz.com.co/apartamento-en-ar...,191344794
4,https://www.fincaraiz.com.co/apartamento-en-ar...,192592371
55,https://www.fincaraiz.com.co/apartamento-en-ar...,192650939
26,https://www.fincaraiz.com.co/apartamento-en-ar...,192835541
64,https://www.fincaraiz.com.co/apartamento-en-ar...,192861055
73,https://www.fincaraiz.com.co/apartamento-en-ar...,192676131
10,https://www.fincaraiz.com.co/apartamento-en-ar...,192737003
40,https://www.fincaraiz.com.co/apartamento-en-ar...,192666097
107,https://www.fincaraiz.com.co/apartamento-en-ar...,192747728


Se elimina la variable original:

In [138]:
datos_apartamentos = datos_apartamentos.drop(columns=['Enlace'])

## Baños

Antes de convertir la variable, se revisan algunos casos al azar buscando identificar características generales:

In [139]:
datos_apartamentos[['Baños']].sample(10, random_state=42)

,Baños
44,2
47,3
4,1
55,3
26,2
64,1
73,2
10,2
40,2
107,1


Se realiza la conversión:

In [140]:
datos_apartamentos['Baños'] = pd.to_numeric(datos_apartamentos['Baños'], errors='coerce').astype('Int64')

Se revisa el comportamiento de la variable:

In [141]:
datos_apartamentos[['Baños']].describe()

,Baños
count,120.0
mean,1.991667
std,0.715916
min,1.0
25%,2.0
50%,2.0
75%,2.0
max,5.0


Como no se ve nada extraño, se deja como está.

## Áreas

Antes de convertir las variables, se revisan algunos casos al azar buscando identificar algo raro:

In [142]:
datos_apartamentos[['Área Construida', 'Área Privada']].sample(10, random_state=42)

,Área Construida,Área Privada
44,49 m2,<NA>
47,200.00 m2,196.00 m2
4,40 m2,35 m2
55,120.00 m2,<NA>
26,75 m2,75 m2
64,90 m2,<NA>
73,106.00 m2,106.00 m2
10,56 m2,50 m2
40,57 m2,<NA>
107,38 m2,38 m2


Como en este caso se especifica la unidad de medida, ésta se incluye en el nombre de las variables y se convierten a numéricas luego de eliminar la parte de texto.

In [143]:
datos_apartamentos['Área Construida (metros cuadrados)'] = datos_apartamentos['Área Construida'].str.extract(r'([\d\.,]+)')[0].str.replace(',', '').astype(float)
datos_apartamentos['Área Privada (metros cuadrados)'] = datos_apartamentos['Área Privada'].str.extract(r'([\d\.,]+)')[0].str.replace(',', '').astype(float)

Se revisa el comportamiento de las variables:

In [144]:
datos_apartamentos[['Área Construida (metros cuadrados)', 'Área Privada (metros cuadrados)']].describe()

,Área Construida (metros cuadrados),Área Privada (metros cuadrados)
count,120.00000,90.000000
mean,193.93900,79.111111
std,904.86672,48.293216
min,1.00000,1.000000
25%,56.00000,56.000000
50%,66.50000,65.000000
75%,90.00000,81.500000
max,8450.00000,300.000000


No parece haber nada raro, por lo que se eliminan las variables originales:

In [145]:
datos_apartamentos = datos_apartamentos.drop(columns=['Área Construida', 'Área Privada'])

## Habitaciones

Antes de convertir la variable, se revisan algunos casos al azar buscando identificar características generales:

In [146]:
datos_apartamentos[['Habitaciones']].sample(10, random_state=42)

,Habitaciones
44,2
47,4
4,2
55,3
26,3
64,3
73,3
10,3
40,2
107,1


Se realiza la conversión:

In [147]:
datos_apartamentos['Habitaciones'] = pd.to_numeric(datos_apartamentos['Habitaciones'], errors='coerce').astype('Int64')

Se revisa el comportamiento de la variable:

In [148]:
datos_apartamentos[['Habitaciones']].describe()

,Habitaciones
count,120.0
mean,2.816667
std,0.840001
min,1.0
25%,2.0
50%,3.0
75%,3.0
max,8.0


Como no hay nada raro, se deja así.

## Parqueaderos

Antes de convertir la variable, se revisan algunos casos al azar buscando identificar características generales:

In [149]:
datos_apartamentos[['Parqueaderos']].sample(10, random_state=42)

,Parqueaderos
44,<NA>
47,<NA>
4,<NA>
55,1
26,<NA>
64,<NA>
73,1
10,1
40,<NA>
107,1


Se realiza la conversión:

In [150]:
datos_apartamentos['Parqueaderos'] = pd.to_numeric(datos_apartamentos['Parqueaderos'], errors='coerce').astype('Int64')

Se revisa el comportamiento de la variable:

In [151]:
datos_apartamentos[['Parqueaderos']].describe()

,Parqueaderos
count,81.0
mean,1.17284
std,0.542912
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,5.0


## Administración

Antes de hacer cualquier conversión, se revisan algunos casos al azar buscando identificar características generales:

In [152]:
datos_apartamentos[['Administración']].dropna().sample(10, random_state=42)

,Administración
54,"$ 212,400.00"
103,$ 330.000
112,"$ 242,000.00"
31,$ 153.000
105,$ 196.000
10,$ 185.000
48,"$ 190,000.00"
115,"$ 239,500.00"
7,$ 151.000
93,$ 466.000


En este caso, se trata de un valor monetario en formato de número latinoamericano (coma como separador decimal y punto como separador de miles) y con signo monetario. Sin embargo, hay casos en que el valor fue puesto en formato anglosajón, por lo que debe tomarse en consideración. De esta forma, se procede a limpiar estas características antes de convertir la variable a numérica:

In [153]:
# eliminación de espacios
datos_apartamentos['Administración'] = datos_apartamentos['Administración'].str.replace(' ', '', regex=False)
# eliminación del signo monetario
datos_apartamentos['Administración'] = datos_apartamentos['Administración'].str.replace('$', '', regex=False)
# identificación de números con formato latinoamericano
datos_apartamentos['Formato ESP'] = datos_apartamentos['Administración'].str.extract(r'^(\d{1,3}\.\d{3}(?:,\d+)?$)', expand=False).notna()
# identificación de números con formato anglosajón
datos_apartamentos['Formato ENG'] = datos_apartamentos['Administración'].str.extract(r'^(\d{1,3},\d{3}(?:\.\d+)?$)', expand=False).notna()
# aplicación de la normalización del formato
datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Administración'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Administración']
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Administración'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Administración']
    .str.replace(',', '', regex=False)
)
# conversion de nulos
datos_apartamentos['Administración'] = datos_apartamentos['Administración'].replace('<NA>', pd.NA)
# conversión de la variable a numérica
datos_apartamentos['Administración'] = pd.to_numeric(datos_apartamentos['Administración'], errors='coerce')
# eliminación de las variables auxiliares
datos_apartamentos = datos_apartamentos.drop(columns=['Formato ESP', 'Formato ENG'])
# revisión de casos
datos_apartamentos[['Administración']].dropna().sample(10, random_state=42)

,Administración
32,281000.0
103,330000.0
92,268950.0
112,242000.0
48,190000.0
115,239500.0
77,104000.0
75,175300.0
94,206400.0
56,220000.0


## Número del piso

En el caso del número de piso, debido a que pisos altos pueden ser más o menos atractivos según el contexto, se convierte la variable a numérica:

In [154]:
datos_apartamentos['Piso N°'] = pd.to_numeric(datos_apartamentos['Piso N°'], errors='coerce').astype('Int64')

Se revisa que haya quedado bien:

In [155]:
datos_apartamentos[['Piso N°']].dropna().sample(10, random_state=42)

,Piso N°
57,12
85,10
56,18
95,5
54,5
80,3
34,1
45,1
18,12
55,4


## Cantidad de pisos

De forma similar a la variable anterior, se convierte a numérica:

In [156]:
datos_apartamentos['Cantidad de pisos'] = pd.to_numeric(datos_apartamentos['Cantidad de pisos'], errors='coerce').astype('Int64')

Se revisa que haya quedado bien:

In [157]:
datos_apartamentos[['Cantidad de pisos']].describe()

,Cantidad de pisos
count,8.0
mean,6.75
std,4.026697
min,3.0
25%,3.75
50%,6.0
75%,8.25
max,15.0


## Administración

Antes de hacer cualquier conversión, se revisan algunos casos al azar buscando identificar características generales:

In [158]:
datos_apartamentos[['Precio']].dropna().sample(10, random_state=42)

,Precio
44,$ 529.000
47,$ 3.000.000
4,$ 616.000
55,$ 1.500.000
26,$ 1.400.000
64,$ 800.000
73,$ 1.950.000
10,$ 615.000
40,$ 1.100.000
107,$ 1.250.000


En este caso, se trata de un valor monetario en formato de número latinoamericano (coma como separador decimal y punto como separador de miles) y con signo monetario. Sin embargo, hay casos en que el valor fue puesto en formato anglosajón, por lo que debe tomarse en consideración. De esta forma, se procede a limpiar estas características antes de convertir la variable a numérica:

In [159]:
# eliminación de espacios
datos_apartamentos['Precio'] = datos_apartamentos['Precio'].str.replace(' ', '', regex=False)
# eliminación del signo monetario
datos_apartamentos['Precio'] = datos_apartamentos['Precio'].str.replace('$', '', regex=False)
# identificación de números con formato latinoamericano
# Generalización para detectar números en formato latinoamericano con millones, miles y decimales
datos_apartamentos['Formato ESP'] = datos_apartamentos['Precio'].str.extract(r'^(\d{1,3}(?:\.\d{3})+(?:,\d+)?|\d+(?:,\d+)?$)', expand=False).notna()
# identificación de números con formato anglosajón
# Generalización para detectar números en formato anglosajón con millones, miles y decimales
datos_apartamentos['Formato ENG'] = datos_apartamentos['Precio'].str.extract(r'^(\d{1,3}(?:,\d{3})+(?:\.\d+)?|\d+(?:\.\d+)?$)', expand=False).notna()
# aplicación de la normalización del formato
datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Precio'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Precio']
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Precio'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Precio']
    .str.replace(',', '', regex=False)
)
# conversion de nulos
datos_apartamentos['Precio'] = datos_apartamentos['Precio'].replace('<NA>', pd.NA)
# conversión de la variable a numérica
datos_apartamentos['Precio'] = pd.to_numeric(datos_apartamentos['Precio'], errors='coerce')
# eliminación de las variables auxiliares
datos_apartamentos = datos_apartamentos.drop(columns=['Formato ESP', 'Formato ENG'])
# revisión de casos
datos_apartamentos[['Precio']].dropna().sample(10, random_state=42)

,Precio
44,529000
47,3000000
4,616000
55,1500000
26,1400000
64,800000
73,1950000
10,615000
40,1100000
107,1250000


# Exportación de la base final

In [160]:
datos_apartamentos[[
    'Identificador', 'Área Construida (metros cuadrados)', 'Área Privada (metros cuadrados)',
    'Habitaciones', 'Baños', 'Estrato', 'Parqueaderos', 'Antigüedad', 'Administración', 'Piso N°',
    'Estado', 'Cantidad de pisos', 'Latitud', 'Longitud', 'Precio'
    ]].to_csv(f"{fecha_actual}_apartamentos_ibague.csv", index=False)